In [2]:
import os
import torch
import numpy as np

In [6]:
print(torch.cuda.is_available())

False


In [7]:
print(torch.backends.mps.is_available())

True


In [8]:
os.cpu_count()

8

In [20]:
data_dir = '/Users/rajkhera/CV-Project/data/'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

In [21]:
def count_img(directory):
    categories = ['0', '1', '2', '3', '4']
    counts = {category: len(os.listdir(os.path.join(directory, category))) for category in categories}
    return counts

train_counts = count_img(train_dir)
val_counts = count_img(val_dir)
test_counts = count_img(test_dir)

### CONFIG FILE

In [29]:
import torch
from albumentations.pytorch import ToTensorV2
import albumentations as A

In [26]:
# Hyperparameter Tuning

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNINGS_RATE = 1e-4
WEIGHT_DECAY = 5e-4
BATCH_SIZE = 128
NUM_EPOCHS = 100
NUM_WORKERS = 6
CHECKPOINT_FILE = "b3.pth.tar"
PIN_MEMORY = True
SAVE_MODEL = True
LOAD_MODEL = True

In [28]:
# Image Augmentations

train_transforms = A.Compose(
    [
        A.Resize(width=150, height=120),
        A.RandomCrop(width=120,height=120),
        A.Normalize(
            mean=[0.3199, 0.2240, 0.1690],
            std=[0.3020, 0.2183, 0.1741],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

val_transforms = A.Compose(
    [
        A.Resize(width=120, height=120),
        A.Normalize(
            mean=[0.3199, 0.2240, 0.1690],
            std=[0.3020, 0.2183, 0.1741],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

### DataSet File

In [33]:
import config
import os 
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm

In [35]:
class DRDataset(Dataset):
    def __init__(self, images_folder, path_to_csv, train=True, transform=None):
        super().__init__()
        self.data = pd.read_csv(path_to_csv)
        self.images_folder = images_folder
        self.image_files = os.listdir(images_folder)
        self.transform = transform
        self.train = train
    
    def __len__(self):
        return self.data.shape[0] if self.train else len(self.image_files)
    
    def __getitem__(self, index):
        if self.train:
            image_file, label = self.data.iloc[index]
        else:
            image_file, label = self.image_files[index], -1
            image_file = image_file.replace(".jpeg", "")

        image = np.array(Image.open(os.path.join(self.images_folder, image_file+".jpeg")))

        if self.transform:
            image = self.transform(image=image)["image"]
        
        return image, label, image_file


In [38]:
if __name__ == "__main__":
    """
    Test if everything works ok
    """
    dataset = DRDataset(
        images_folder="../train/images_resized_650/",
        path_to_csv="../train/trainLabels.csv",
        transform=config.val_transforms,
    )
    loader = DataLoader(
        dataset=dataset, batch_size=32, num_workers=2, shuffle=True, pin_memory=True
    )

    for x, label, file in tqdm(loader):
        print(x.shape)
        print(label.shape)
        import sys
        sys.exit()

AttributeError: module 'config' has no attribute 'val_transforms'